<a href="https://colab.research.google.com/github/arsalanyaghoobi/Text_Embedding--Jason-Sarcasm-Data_set/blob/master/json_sarcasm_datastore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
vocab_size=10000
embedding_dim=16
max_length=32
oov_token="<oov>"
trunc_type='post'
padding_type='post'
training_size=20000

In [4]:
!wget __no_check_certificate \
https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json

--2020-07-30 08:01:16--  http://__no_check_certificate/
Resolving __no_check_certificate (__no_check_certificate)... failed: Name or service not known.
wget: unable to resolve host address ‘__no_check_certificate’
--2020-07-30 08:01:16--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.97.128, 108.177.125.128, 74.125.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘sarcasm.json’

sarcasm.json        100%[===================>]   5.38M  --.-KB/s    in 0.09s   

2020-07-30 08:01:17 (62.7 MB/s) - ‘sarcasm.json’ saved [5643545/5643545]

FINISHED --2020-07-30 08:01:17--
Total wall clock time: 0.5s
Downloaded: 1 files, 5.4M in 0.09s (62.7 MB/s)


In [5]:
with open ('/content/sarcasm.json','r') as f:
  datastore=json.load(f)

sentences=[]
labels=[]

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [6]:
training_sentences=sentences[0:training_size]
test_sentences=sentences[training_size:]
training_labels=labels[0:training_size]
test_labels=labels[training_size:]

In [7]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_token)
word_index=tokenizer.word_index

In [8]:
tokenizer.fit_on_texts(training_sentences)
training_sequences=tokenizer.texts_to_sequences(training_sentences)
training_padded=pad_sequences(training_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [9]:
test_sequences=tokenizer.texts_to_sequences(test_sentences)
test_padded=pad_sequences(test_sequences,maxlen=max_length,padding=padding_type,truncating=trunc_type)

In [10]:
training_labels_final=np.array(training_labels)
test_labels_final=np.array(test_labels)

In [11]:
import tensorflow as tf
model=tf.keras.Sequential([
                        tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
                        tf.keras.layers.GlobalAveragePooling1D(),
                        tf.keras.layers.Dense(50,activation='relu'),
                        tf.keras.layers.Dense(40,activation='relu'),
                        tf.keras.layers.Dense(20,activation='relu'),
                        tf.keras.layers.Dense(10,activation='relu'),
                        tf.keras.layers.Dense(1,activation='sigmoid')
                         ])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 32, 16)            160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                850       
_________________________________________________________________
dense_1 (Dense)              (None, 40)                2040      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                820       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 1

In [13]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [14]:
history=model.fit(training_padded,training_labels_final,
                  validation_data=(test_padded,test_labels_final),
                  epochs=20,verbose=2)

Epoch 1/20
625/625 - 2s - loss: 0.4792 - accuracy: 0.7501 - val_loss: 0.3513 - val_accuracy: 0.8480
Epoch 2/20
625/625 - 2s - loss: 0.2567 - accuracy: 0.8949 - val_loss: 0.3512 - val_accuracy: 0.8514
Epoch 3/20
625/625 - 2s - loss: 0.1916 - accuracy: 0.9255 - val_loss: 0.3818 - val_accuracy: 0.8429
Epoch 4/20
625/625 - 2s - loss: 0.1472 - accuracy: 0.9469 - val_loss: 0.4156 - val_accuracy: 0.8438
Epoch 5/20
625/625 - 2s - loss: 0.1164 - accuracy: 0.9603 - val_loss: 0.4720 - val_accuracy: 0.8408
Epoch 6/20
625/625 - 2s - loss: 0.0960 - accuracy: 0.9674 - val_loss: 0.5688 - val_accuracy: 0.8219
Epoch 7/20
625/625 - 2s - loss: 0.0760 - accuracy: 0.9756 - val_loss: 0.5984 - val_accuracy: 0.8337
Epoch 8/20
625/625 - 2s - loss: 0.0619 - accuracy: 0.9808 - val_loss: 0.6369 - val_accuracy: 0.8280
Epoch 9/20
625/625 - 2s - loss: 0.0565 - accuracy: 0.9819 - val_loss: 0.6810 - val_accuracy: 0.8292
Epoch 10/20
625/625 - 2s - loss: 0.0453 - accuracy: 0.9863 - val_loss: 0.7818 - val_accuracy: 0.8232

In [15]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string,'val_'+string])
  plt.show()

plot_graphs(history, "accurcy")
plot_graphs(history, "loss")

KeyError: ignored